In [1]:
(do 
    (require `serpent.core)
    (require `serpent.pprint)
    (require `serpent.jupyter)
    (serpent.core/configure)
    (add-dependencies [org.mitre.tdp/ttfs-workflow "3.1.28" :exclusions [[net.sf.jsi/jsi]]])
    (set-cda-config :root "tdp:3.1"))

Retrieving org/mitre/tdp/ttfs-workflow/3.1.28/ttfs-workflow-3.1.28.pom (30k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-releases/
Retrieving org/mitre/tdp/tdp-parent/3.1.28/tdp-parent-3.1.28.pom (13k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-releases/
Retrieving org/mitre/cda/cda-parent/1.4.50/cda-parent-1.4.50.pom (14k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-releases/
Retrieving org/mitre/cda/cda-engine/1.4.50/cda-engine-1.4.50.pom (3k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-releases/
Retrieving org/mitre/cda/cda-core/1.4.50/cda-core-1.4.50.pom (3k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-releases/
Retrieving org/mitre/tdp/tdp-trajectory/3.1.28/tdp-trajectory-3.1.28.pom (20k)
    from https://dali.mitre.org/nexus/content/repositories/mitre-caasd-releases/
Retrieving org/mitre/cda/cda-workflow/1.4.50/cda-workflow-1.4.50.pom (4k)
    fr

{:configuration
 #object[org.apache.hadoop.conf.Configuration 0xdcd5740 "\"Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml\""],
 :meta-repository
 #object[org.mitre.cda.hbase.HBaseMetaRepository 0x6b3a2251 "\"HBaseMetaRepository[tdp:3.1-meta]\""],
 :hbase-repository
 #object[org.mitre.cda.core.CachingKeyDereferencer 0x7757b7a3 "\"org.mitre.cda.core.CachingKeyDereferencer@7757b7a3\""]}


In [2]:
(add-dependencies [org.mitre.tdp/ttfs-procedure "3.1.28" :exclusions [[net.sf.jsi/jsi]]])
(add-dependencies [org.mitre.tdp/tdp-infrastructure "3.1.28" :exclusions [[net.sf.jsi/jsi]]])

("com.google.guava:guava:23.0"
 "net.java.dev.jets3t:jets3t:0.9.4"
 "org.json:json:20140107"
 "org.apache.commons:commons-math3:3.4.1"
 "org.codehaus.mojo:animal-sniffer-annotations:1.14"
 "com.google.errorprone:error_prone_annotations:2.0.18"
 "org.codehaus.jackson:jackson-xc:1.9.13"
 "org.apache.commons:commons-math:2.2"
 "org.scala-lang:scala-reflect:2.11.8"
 "org.apache.httpcomponents:httpcore:4.4.1"
 "com.google.j2objc:j2objc-annotations:1.1"
 "commons-cli:commons-cli:1.2"
 "com.google.code.findbugs:jsr305:1.3.9"
 "org.objenesis:objenesis:2.1"
 "joda-time:joda-time:2.9.4"
 "com.jamesmurty.utils:java-xmlbuilder:1.1"
 "net.iharder:base64:2.3.8"
 "commons-logging:commons-logging:1.2"
 "commons-codec:commons-codec:1.9"
 "org.scala-lang:scala-library:2.11.8"
 ...)


In [4]:
(import org.mitre.caasd.ttfs.ArincTransition
        org.mitre.caasd.ttfs.ArincFix
        org.mitre.caasd.ttfs.ArincAirway
        org.mitre.caasd.ttfs.ArincAirport)

org.mitre.caasd.ttfs.ArincAirport


In [5]:
;; data is small we can just materialize an entire cycle
(def cyc 20210101)

(def transitions (load-data ArincTransition :dates cyc :only "CIFP"))
(def fixes (load-data ArincFix :dates cyc :only "CIFP"))
(def airways (load-data ArincAirway :dates cyc :only "CIFP"))
(def airports (load-data ArincAirport :dates cyc :only "CIFP"))

(str "Transitions: " (count transitions) " Fixes: " (count fixes) " Airways: " (count airways) " Airports: " (count airports))

"Transitions: 61338 Fixes: 67911 Airways: 1550 Airports: 13765"


### Helper Functions

In [18]:
;; Grab all the transitions
(defn null-or-equals [e v]
    (or (= nil v) (= e nil) (= e v)))

(defn proc-filter [pname & kwargs]
    (let [{:keys [airport source] :or {airport nil source nil} :as opts} kwargs]
        (fn [p] (and 
                    (null-or-equals airport ((comp :name :airport) p)) 
                    (null-or-equals pname (:procedure_name p))
                    (null-or-equals source (:navigation_source p))))))

(defn fix-filter [col]
    (fn [f] 
        (some #(= % ((comp :name :identification) f)) col)))

(defn airway-filter [awname & kwargs]
    (let [{:keys [entry exit area-code source]
           :or {entry nil
                exit nil
                area-code nil
                source nil}
           :as opts}
          kwargs]
        (fn [a] (and
                    (null-or-equals awname ((comp :name :identification) a))
                    (some #(null-or-equals entry ((comp :name :identification :path_terminator) %)) (:points a))
                    (some #(null-or-equals exit ((comp :name :identification :path_terminator) %)) (:points a))
                    (null-or-equals area-code ((comp :customer_area_code :arinc_record_info) a))
                    (null-or-equals source ((comp :source :arinc_record_info) a))))))

(defn peek? [ele f] (f ele) ele)

(defn cda-key->string [k] (.toString (.getPrimaryKey k)))

(defn fixName [leg] (str ((comp :name :identification :path_terminator) leg)))
(defn fixLatLon [leg] (let [latitude ((comp :latitude :path_terminator) leg)
                            longitude ((comp :longitude :path_terminator) leg)]
                          (str " , " latitude " , " longitude)))
(defn toMockitoAirwayBody [airway]
    (let [body (reduce str (->> (:legs airway)
     (map #(str "Leg " (fixName %) "= TF( \"" (fixName %) "\"" (fixLatLon %) "); \n"))))]
    (str body " - primary_key " (cda-key->string (:primary_key airway)) "\n")))

#'user/toMockitoAirwayBody


### Airway Mocking

In [6]:
(def airwayy (->> airways (filter #(= "Q6" (:identifier %)))))

#'user/airwayy


In [8]:
(->> airwayy (map toMockitoAirwayBody))

("Leg TKA= TF( "TKA" , 62.29837777777777 , -150.10524999999998); 
Leg JOKAP= TF( "JOKAP" , 63.91291111111111 , -150.97479166666668); 
Leg KUTDE= TF( "KUTDE" , 66.32227499999999 , -152.48358055555553); 
Leg LACIL= TF( "LACIL" , 69.504975 , -155.00948055555557); 
Leg BRW= TF( "BRW" , 71.27342777777778 , -156.78811666666667); 
 - primary_key 4-2YeCFS_9AAAJZ
")


### Transition Mocking

#### Procedures By Airport

In [7]:
(def airport-transitions (->> transitions (filter #(= "KSEA" ((comp :name :airport) %)))))
(frequencies (map :procedure_name airport-transitions))

{"H34CZ" 4,
 "I34R" 5,
 "L16L" 6,
 "EPH8" 4,
 "SEA7" 6,
 "L16C" 6,
 "JAWBN6" 6,
 "HAWKZ7" 8,
 "ISBRG1" 4,
 "H34RZ" 3,
 "CHINS4" 6,
 "H16CZ" 6,
 "R16RY" 4,
 "I34C" 4,
 "R34CY" 4,
 "OLM2" 5,
 "R16LY" 4,
 "MARNR7" 8,
 "L34L" 4,
 "MONTN1" 6,
 "OZWLD1" 4,
 "I16C" 6,
 "I16L" 6,
 "SUMMA2" 8,
 "L34R" 5,
 "R34RY" 4,
 "GLASR1" 8,
 "R16CY" 4,
 "H34LZ" 4,
 "L16R" 6,
 "I34L" 4,
 "H16LZ" 4,
 "H16RZ" 6,
 "R34LY" 4,
 "JEFPO1" 4,
 "HAROB6" 10,
 "BANGR9" 8,
 "I16R" 6,
 "L34C" 4,
 "ELMAA3" 6}


In [8]:
(def proc-trans (filter (proc-filter "SUMMA2" :airport "KSEA" :source "CIFP") transitions))
(str "total transitions: " (count proc-trans))

"total transitions: 8"


In [9]:
(clojure.string/join "," (map (comp str get-key :primary_key) proc-trans))

"4mIH87B6x_AANyt,4mIH87B6x_AANyu,4mIH87B6x_AANyv,4mIH87B6x_AANyw,4mIH87B6x_AANyx,4mIH87B6x_AANyy,4mIH87B6x_AANyz,4mIH87B6x_AANy0"


In [10]:
(map :primary_key (filter #(= "KSEA" ((comp :name :identification) %)) airports))

(Key<4pvX1Gw_G4AACbj>)


In [11]:
(first (load-data org.mitre.caasd.ttfs.NfdcAirport :dates cyc :columns [:primary_key [:icao_identifier = "KSEA"] :ref_point_latitude :ref_point_longitude]))

Record
{:primary_key Key<4Fk9t23D4DAABNw>,
 :icao_identifier "KSEA",
 :ref_point_latitude 47.449889,
 :ref_point_longitude -122.311778}


#### Mocking That Business

In [19]:
(defn toMockitoTransitionBody [transition]
    (reduce str (->> (:legs transition)
     (map #(str "Leg " (fixName %) "=" (:leg_type %) " ( \"" (fixName %) "\"" (fixLatLon %) "); \n")))))

(defn toMockitoTransitionMethod [transition] 
    (let [tname (:transition_name transition)
          pname (:procedure_name transition)
          aname ((comp :name :airport) transition)]
        (str "private static Transition " (:transition_name transition) "(){\n"
         (toMockitoTransitionBody transition)
         "return transition(\"" tname "\",\"" pname "\",\"" aname "\",,,);\n}")))

(defn mockitizeMeCaptain [transitions]
    (reduce str (->> transitions (map #(str (toMockitoTransitionMethod %) "\n")))))

#'user/mockitizeMeCaptain


In [20]:
(mockitizeMeCaptain proc-trans)

"private static Transition RW34C(){
Leg =VI ( "" ,  , ); 
Leg NEZUG=CF ( "NEZUG" , 47.56866666666667 , -122.30966666666666); 
Leg =VM ( "" ,  , ); 
return transition("RW34C","SUMMA2","KSEA",,,);
}
private static Transition RW34L(){
Leg =VI ( "" ,  , ); 
Leg NEZUG=CF ( "NEZUG" , 47.56866666666667 , -122.30966666666666); 
Leg =VM ( "" ,  , ); 
return transition("RW34L","SUMMA2","KSEA",,,);
}
private static Transition RW16L(){
Leg =VI ( "" ,  , ); 
Leg NEVJO=CF ( "NEVJO" , 47.25216666666667 , -122.30966666666666); 
Leg =VI ( "" ,  , ); 
Leg SUMMA=CF ( "SUMMA" , 46.61785833333333 , -121.98832222222222); 
return transition("RW16L","SUMMA2","KSEA",,,);
}
private static Transition LKV(){
Leg SUMMA=IF ( "SUMMA" , 46.61785833333333 , -121.98832222222222); 
Leg LKV=TF ( "LKV" , 42.492844444444444 , -120.50710555555555); 
return transition("LKV","SUMMA2","KSEA",,,);
}
private static Transition RW16C(){
Leg =VI ( "" ,  , ); 
Leg NEVJO=CF ( "NEVJO" , 47.25216666666667 , -122.30966666666666); 
Leg =